In [1]:
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

import sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_csv("emails.csv", usecols = ['text', 'spam'])

In [3]:
skips = [".", ",", ":", ";", "'", '"', '_', '-', '%', '_', '(', ')', '!', '/', '--', '?', '$', '|', 
         '=', '*', '@', '<', '>', '^', '{', '}', '`', '~', '+', ']', '[' '\\', '\\\\', "&", '#',
         '\x01', '\x05', '\x06', '\x07', '\x0f', '\x14', '\x15']

for i in range(len(data['text'])):
    data['text'][i].lower()
    
    for ch in skips:
        data['text'][i] = data['text'][i].replace(ch, "")
        
    data['text'][i] = re.sub(r'\w*\d\w*', '', data['text'][i]).strip()
        
    data['text'][i] = word_tokenize(data['text'][i])

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_7540/2369810091.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'][i] = data['text'][i].replace(ch, "")
C:\Users\LENOVO\AppData\Local\Temp/ipykernel_7540/2369810091.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'][i] = re.sub(r'\w*\d\w*', '', data['text'][i]).strip()
C:\Users\LENOVO\AppData\Local\Temp/ipykernel_7540/2369810091.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [4]:
for i in range(len(data['text'])):
    data['text'][i] = [di for di in data['text'][i] if not di in set(stopwords.words('english'))]

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_7540/679345973.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'][i] = [di for di in data['text'][i] if not di in set(stopwords.words('english'))]


In [5]:
for i in range(len(data['text'])):
    ps = PorterStemmer()
    data['text'][i] = [ps.stem(di) for di in data['text'][i]]

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_7540/2137653994.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'][i] = [ps.stem(di) for di in data['text'][i]]


In [6]:
vocab = []

for i in range(len(data['text'])):
    for j in range(len(data['text'][i])):
        if data['text'][i][j] not in vocab:
            vocab.append(data['text'][i][j])
    
print(len(vocab))

25592


In [7]:
word_mt = np.zeros([len(data['text']), len(vocab)])

for i in range(len(data['text'])):
    for j in range(len(data['text'][i])):
        tmp = vocab.index(data['text'][i][j])
        word_mt[i][tmp] = 1

print(word_mt)

[[1. 1. 1. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 1. 1.]]


In [8]:
X = word_mt
y = data['spam']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [9]:
bnl = BernoulliNB()
bnl.fit(X_train, y_train)
y_pred = bnl.predict(X_test)
print("Accuracy Bernoulli: ", metrics.accuracy_score(y_test, y_pred))

Accuracy Bernoulli:  0.979050279329609


In [10]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print("Accuracy Gaussian: ", metrics.accuracy_score(y_test, y_pred))

Accuracy Gaussian:  0.9685754189944135


In [11]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)
print("Accuracy Multinomial: ", metrics.accuracy_score(y_test, y_pred))

Accuracy Multinomial:  0.9881284916201117
